In [ ]:
# 01_data_visualization.ipynb

# Import necessary libraries
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from typing import Optional


def load_locations(filepath: str) -> pd.DataFrame:
    """
    Load delivery locations from a CSV file.

    Args:
        filepath (str): Path to the CSV file containing delivery locations.

    Returns:
        pd.DataFrame: DataFrame with delivery locations.
    """
    locations_df = pd.read_csv(filepath)
    # Validate required columns exist
    required_columns = {"id", "latitude", "longitude", "address"}
    if not required_columns.issubset(locations_df.columns):
        raise ValueError(f"CSV file must contain columns: {required_columns}")
    return locations_df


def create_map(
    locations: pd.DataFrame, 
    map_center: Optional[list[float]] = None, 
    zoom_start: int = 12
) -> folium.Map:
    """
    Create a folium map with delivery location markers clustered.

    Args:
        locations (pd.DataFrame): DataFrame containing 'latitude', 'longitude', and 'address'.
        map_center (Optional[list[float]]): Latitude and longitude for initial map center.
                                            If None, center will be mean of locations.
        zoom_start (int): Initial zoom level for the map.

    Returns:
        folium.Map: Folium map object with delivery markers.
    """
    if map_center is None:
        map_center = [locations['latitude'].mean(), locations['longitude'].mean()]

    delivery_map = folium.Map(location=map_center, zoom_start=zoom_start)

    marker_cluster = MarkerCluster().add_to(delivery_map)

    for _, row in locations.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"ID: {row['id']}<br>Address: {row['address']}",
            tooltip=row['address'],
            icon=folium.Icon(color='blue', icon='truck', prefix='fa')
        ).add_to(marker_cluster)

    return delivery_map


# Main execution
if __name__ == "__main__":
    # Load locations data
    locations_path: str = "data/locations.csv"
    locations_df: pd.DataFrame = load_locations(locations_path)

    # Create and display the map
    delivery_map: folium.Map = create_map(locations_df)
    delivery_map.save("output/delivery_locations_map.html")

    print("Map saved to output/delivery_locations_map.html")

